In [18]:
x = np.arange(512*512).reshape(512,512)
print(x[128:384,128:384].shape)

(256, 256)


In [1]:
import os
import torch
import numpy as np
import imageio
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from time import time as time

from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import torchnet as tnt
import functools

import mock
from tqdm import notebook as tqdm
from distutils.dir_util import copy_tree


# GLOBAL SETTINGS
PlotSize = 12                                     # Size of plots
matplotlib.rcParams['figure.figsize'] = [PlotSize*2, PlotSize]  
CMAP = matplotlib.colors.ListedColormap(['black', 'white', 'orange'])               # Color mapping 
np.set_printoptions(precision=2, suppress=True)  # Array print precision

# CLASS AND FEATURE DESCRIPTION
class_names = ['BACKGRD','PINUS','PICEA']

# PATHS TO TRAIN/TEST DATA
data_path = '../data/66_33/'
training_set_path = data_path + 'train/'         # Relative path to training patch root folder
test_set_path =     data_path + '/test/'          # Relative path to test patch root folder

num_of_training_tiles = len(os.listdir(training_set_path + 'GT/'))
num_of_test_tiles = len(os.listdir(test_set_path + 'GT/'))

# USE CIR OR RGB DATA
use_cir = True
use_rgb = True

# MODEL NAME... USED AS FILENAME OF SAVED MODEL AND FOR APPROPRIATE RESULTS FOLDER
model_name = 'U_Net_66_33'

In [ ]:
def read_patch(root_folder, cir, rgb, gt=True):
    ##########################################################
    # READ IMAGES as FLOAT
    
    if cir:
        cir_file_list = os.listdir(root_folder + 'CIR/')
        cir_list = []
        
        for file in cir_file_list:
            cir_patch = imageio.imread(root_folder + 'CIR/' + file).astype(np.float32)
            cir_patch = cir_patch[:,:,:].transpose([2,0,1])
            cir_patch = cir_patch * 1/255
            
            cir_list.append(cir_patch)
            del cir_patch

        cir_features = np.stack(cir_list, axis=0)    
    
    if rgb:
        rgb_file_list = os.listdir(root_folder + 'RGB/')
        rgb_list = []
        
        for file in rgb_file_list:
            rgb_patch = imageio.imread(root_folder + 'RGB/' + file).astype(np.float32)
            rgb_patch = rgb_patch[:,:,:].transpose([2,0,1])
            rgb_patch = rgb_patch * 1/255
            
            rgb_list.append(rgb_patch)
            
            del rgb_patch
        
        rgb_features = np.stack(rgb_list, axis=0)

    if cir and rgb:
        features = np.concatenate([cir_features, rgb_features], axis=1)
    elif cir:
        features = cir_features
    elif rgb:
        features = rgb_features
    else:
        print('No valid data input.')
    features = torch.from_numpy(features)
    
    
    if gt:
        gt_file_list = os.listdir(root_folder + 'GT/')
        gt_list = []

        for file in gt_file_list:
            gt_patch = imageio.imread(root_folder + 'GT/' + file).astype(np.int64)
 
            gt_list.append(gt_patch[:,:])
            del gt_patch

        ground_truth = np.stack(gt_list, axis=0)
        ground_truth = torch.from_numpy(ground_truth)
    
    if gt:
        return features, ground_truth
    else:
        return features

In [ ]:
class UNet(nn.Module):
    """
    U-Net network for semantic segmentation
    """
  
    def __init__(self, n_channels, encoder_conv_width, decoder_conv_width, n_class, cuda):
        """
        initialization function
        n_channels, int, number of input channel
        encoder_conv_width, int list, size of the feature maps of convs for the encoder
        decoder_conv_width, int list, size of the feature maps of convs for the decoder
        n_class = int,  the number of classes
        """
        super(UNet, self).__init__() #necessary for all classes extending the module class
    
        self.maxpool=nn.MaxPool2d(2,2,return_indices=False) #maxpooling layer
    
        #encoder
        self.c1 = nn.Sequential(nn.Conv2d(n_channels,encoder_conv_width[0],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c2 = nn.Sequential(nn.Conv2d(encoder_conv_width[0],encoder_conv_width[1],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c3 = nn.Sequential(nn.Conv2d(encoder_conv_width[1],encoder_conv_width[2],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c4 = nn.Sequential(nn.Conv2d(encoder_conv_width[2],encoder_conv_width[3],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c5 = nn.Sequential(nn.Conv2d(encoder_conv_width[3],encoder_conv_width[4],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c6 = nn.Sequential(nn.Conv2d(encoder_conv_width[4],encoder_conv_width[5],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c7 = nn.Sequential(nn.Conv2d(encoder_conv_width[5],encoder_conv_width[6],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c8 = nn.Sequential(nn.Conv2d(encoder_conv_width[6],encoder_conv_width[7],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c9 = nn.Sequential(nn.Conv2d(encoder_conv_width[7],encoder_conv_width[8],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c10 = nn.Sequential(nn.Conv2d(encoder_conv_width[8],encoder_conv_width[9],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        #decoder
        self.c11 = nn.ConvTranspose2d(encoder_conv_width[9], int(decoder_conv_width[0]/2),kernel_size=2, stride=2)
        self.c12 = nn.Sequential(nn.Conv2d(decoder_conv_width[0],decoder_conv_width[1],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c13 = nn.Sequential(nn.Conv2d(decoder_conv_width[1],decoder_conv_width[2],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c14 = nn.ConvTranspose2d(decoder_conv_width[2], int(decoder_conv_width[3]/2),kernel_size=2, stride=2)
        self.c15 = nn.Sequential(nn.Conv2d(decoder_conv_width[3],decoder_conv_width[4],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c16 = nn.Sequential(nn.Conv2d(decoder_conv_width[4],decoder_conv_width[5],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c17 = nn.ConvTranspose2d(decoder_conv_width[5], int(decoder_conv_width[6]/2),kernel_size=2, stride=2)
        self.c18 = nn.Sequential(nn.Conv2d(decoder_conv_width[6],decoder_conv_width[7],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c19 = nn.Sequential(nn.Conv2d(decoder_conv_width[7],decoder_conv_width[8],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c20 = nn.ConvTranspose2d(decoder_conv_width[8], int(decoder_conv_width[9]/2),kernel_size=2, stride=2)
        self.c21 = nn.Sequential(nn.Conv2d(decoder_conv_width[9],decoder_conv_width[10],3,padding=1, padding_mode='reflect'),nn.ReLU(True))
        self.c22 = nn.Sequential(nn.Conv2d(decoder_conv_width[10],decoder_conv_width[11],3,padding=1, padding_mode='reflect'),nn.ReLU(True)) 
        
        #final classifying layer
        self.classifier=nn.Conv2d(decoder_conv_width[11],n_class,1,padding=0)

        #weight initialization

        self.c1[0].apply(self.init_weights)
        self.c2[0].apply(self.init_weights)
        self.c3[0].apply(self.init_weights)
        self.c4[0].apply(self.init_weights)
        self.c5[0].apply(self.init_weights)
        self.c6[0].apply(self.init_weights)
        self.c7[0].apply(self.init_weights)
        self.c8[0].apply(self.init_weights)
        self.c9[0].apply(self.init_weights)
        self.c10[0].apply(self.init_weights)
        
        self.c12[0].apply(self.init_weights)
        self.c13[0].apply(self.init_weights)
        
        self.c15[0].apply(self.init_weights)
        self.c16[0].apply(self.init_weights)
        
        self.c18[0].apply(self.init_weights)
        self.c19[0].apply(self.init_weights)
        
        self.c21[0].apply(self.init_weights)
        self.c22[0].apply(self.init_weights)
        self.classifier.apply(self.init_weights)
    
        if cuda: #put the model on the GPU memory
            self.cuda()
    
    def init_weights(self,layer): #gaussian init for the conv layers
        nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
    
    def forward(self,input):
        """
        the function called to run inference
        """  
        #encoder
        #level 1
        x1 = self.c2(self.c1(input))
        x2 = self.maxpool(x1)
        #level 2
        x3 = self.c4(self.c3(x2))
        del x2
        x4 = self.maxpool(x3)
        #level 3
        x5 = self.c6(self.c5(x4))
        del x4
        x6 = self.maxpool(x5)
        #Level 4
        x7 = self.c8(self.c7(x6))
        del x6
        x8 = self.maxpool(x7)
        #Level 5
        x9 = self.c10(self.c9(x8))
        del x8
        #decoder
        #Level 4
        y8 = torch.cat((self.c11(x9),x7),1)
        del x7,x9
        y7 = self.c13(self.c12(y8))
        del y8
        #Level 3
        y6 = torch.cat((self.c14(y7),x5),1)
        del x5,y7
        y5 = self.c16(self.c15(y6))
        del y6
        #level 2
        y4 = torch.cat((self.c17(y5),x3),1)
        del x3, y5
        y3 = self.c19(self.c18(y4))
        del y4
        #level 1       
        y2 = torch.cat((self.c20(y3),x1),1)
        del x1, y3
        y1 = self.c22(self.c21(y2))
        del y2
        #output         
        out = self.classifier(y1)
    
        return out

In [ ]:
# Path to the state_dictionary
state_dict_path = 'trained_models/U_Net_66_33.pt'

In [ ]:
# Parameters for model definition
args = mock.Mock() #stores the parameters

args.n_class = len(class_names)
args.n_channel = 6 if use_cir and use_rgb else 3
args.conv_width = [64,64,128,128,256,256,512,512,1024,1024]
args.dconv_width = [1024,512,512,512,256,256,256,128,128,128,64,64]
args.cuda = False

In [ ]:
# Load a trained model state_dictionary
model = UNet(args.n_channel, args.conv_width, args.dconv_width, args.n_class, args.cuda)
model.load_state_dict(torch.load(state_dict_path))
model.eval()

In [ ]:
def classify_and_export(model_b, in_features_b, results_path_b, gpu):
    i = 0
    for patch in os.listdir(results_path_b):
        in_patch = in_features_b[i,:,:,:]
        if gpu:
            pred = model_b(in_patch[None,:,:,:].cuda()).cpu().detach().numpy()
        else:
            pred = model_b(in_patch[None,:,:,:]).detach().numpy()
        pred = pred[0,:,:,:].argmax(0).squeeze()

        imageio.imwrite(results_path_b + patch, pred.astype(np.uint8))
        i+=1

In [ ]:
source_paths = ['../data/2012/western/transects/']
results_paths = ['../data/2012/western/transects/results/']

In [ ]:
i = 0
for source_path in source_paths:
    a = time()
    in_features = read_patch(source_path, use_cir, use_rgb, gt=False)
    b = time()
    print('Dataset loaded in ' + str(b-a) + ' s')
    
    if use_rgb:
        copy_tree(source_path + 'RGB/', results_paths[i], update=1)
    elif use_cir:
        copy_tree(source_path + 'CIR/', results_paths[i], update=1)
    else:
        print('no input files')
    c = time()
    classify_and_export(model, in_features, results_paths[i], args.cuda)
    d = time()
    print('Classified and saved in ' + str(d-c) + ' s')
    del in_features
    i+=1